# Prediction of bile predicaments

In [ ]:
import pandas as pd

import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_decision_forests as tfdf

from sklearn.model_selection import train_test_split

import tensorflow_decision_forests as tfdf

import dtreeviz

from matplotlib import pyplot as plt
from IPython import display

# avoid "Arial font not found warnings"
import logging
logging.getLogger('matplotlib.font_manager').setLevel(level=logging.CRITICAL)

display.set_matplotlib_formats('retina') # generate hires plots

np.random.seed(1234)  # reproducible plots_data for explanatory reasons

## Get & clean data

### Load and inspect

In [ ]:
df_idc = pd.read_excel("data/Intraductal_concrement.xlsx", sheet_name=0, skiprows=1, header=None)
df_idc_visits = pd.read_excel("data/Intraductal_concrement.xlsx", sheet_name=0, skiprows=0, header=None)
df_idc.columns = df_idc_visits.iloc[0] + '_' + df_idc.iloc[0]
df_idc.columns = df_idc.columns.str.lower()
df_idc.insert(2, 'has_idc', "1")
df_idc.drop(index=0, inplace=True)

In [ ]:
df_idc.iloc[:, 2:]

In [ ]:
df_idc.columns.tolist()[:10]

In [ ]:
df_no_idc = pd.read_excel("data/No_intraductal_concrement.xlsx", sheet_name=0, skiprows=1, header=None)
df_no_idc_visits = pd.read_excel("data/No_intraductal_concrement.xlsx", sheet_name=0, skiprows=0, header=None)
df_no_idc.columns = df_no_idc_visits.iloc[0] + '_' + df_no_idc.iloc[0]
df_no_idc.columns = df_no_idc.columns.str.lower()
df_no_idc.insert(2, 'has_idc', "0")
df_no_idc.drop(index=0, inplace=True)

In [ ]:
df_no_idc.iloc[:, 2:]

In [ ]:
df_no_idc.columns.tolist()[:10]

### Check that column names are identical

In [ ]:
assert(df_idc.columns.tolist() == df_no_idc.columns.tolist())

### Join the data frames

In [ ]:
input_data = pd.concat([df_idc, df_no_idc])

### Clean and format

In [ ]:
input_data.replace(['unknown', 'missing', 'ns', ' ns', "ms", "s", "sn"], np.nan, inplace=True)
lab_cols = input_data.filter(like='lab_values').columns
input_data[lab_cols] = input_data[lab_cols].replace(',', '.', regex=True)
input_data[lab_cols] = input_data[lab_cols].replace(['.*\\.\\..*', '.*\\.\\.\\..*'], pd.NA, regex=True)
input_data[lab_cols] = input_data[lab_cols].replace('ß', '0', regex=True)
input_data[lab_cols] = input_data[lab_cols].replace('.', np.nan)
input_data[lab_cols] = input_data[lab_cols].replace('n2.23', np.nan)

In [ ]:
input_data[['ercp_date', 'baseline_characteristics_admission_date']] = input_data[['ercp_date', 'baseline_characteristics_admission_date']].replace(['.*\\.\\..*', '.*\\.\\.\\..*'], pd.NA, regex=True)

input_data = input_data.convert_dtypes()
input_data[lab_cols] = input_data[lab_cols].astype('float64')

# Add column if ercp_date was within 3 days of admission baseline_characteristics_admission_date
input_data['ercp_date'] = pd.to_datetime(input_data['ercp_date'])
input_data['baseline_characteristics_admission_date'] = pd.to_datetime(input_data['baseline_characteristics_admission_date'])
input_data['ercp_date_minus_admission_date'] = input_data['ercp_date'] - input_data['baseline_characteristics_admission_date']
input_data['ecrp_gte_3_days_naT'] = input_data['ercp_date_minus_admission_date'] >= pd.Timedelta(days=3) 
input_data['ercp_date_na'] = input_data['ercp_date'].isna()
input_data['ecrp_gte_3_days'] =  input_data['ercp_date_na'] | input_data['ecrp_gte_3_days_naT']

# Rename column names of input_data
input_data.columns = input_data.columns.str.replace('/', '_')
input_data.columns = input_data.columns.str.replace(' ', '')

In [ ]:

input_data.iloc[:, 2:]

### Write data to csv for external analysis

In [ ]:
input_data_tab1 = input_data.copy()
input_data_tab1.loc[:, 'has_idc'] = input_data_tab1['has_idc'].astype('category')
input_data_tab1.to_csv("data/input_data_tab1.csv", index=False)

## Prepare data for TFDF: BASELINE

### Features

In [ ]:
TARGET_COLUMN_NAME = ["has_idc"]

ECRP_GTE_3_DAYS = ["ecrp_gte_3_days"]

DATA_HEADER = input_data.columns

CATEGORICAL_FEATURE_NAMES = [
                            'baseline_characteristics_sex', 
                            'baseline_characteristics_nicotine_use',
                            #'baseline_characteristics_alcohol_use',
                            ]

NUMERIC_BASELINE_FEATURE_NAMES = [
                            'baseline_characteristics_age',
                            'baseline_characteristics_height_in_cm',
                            'baseline_characteristics_weight_in_kg',
                            # 'baseline_characteristics_bmi',
                             #'eus_ercp_mrcp_or_as_dhc_width_in_mm'
                            ]

# NUMERIC_LAB_FEATURE_NAMES_ADMISSION = input_data.filter(like='lab_values_admission').columns.tolist()
# Remove variables with mostly missing values
NUMERIC_LAB_FEATURE_NAMES_ADMISSION =['lab_values_admission_crp_in_mg_dl',
                                    #'lab_values_admission_pct_in_ng_ml',
                                    #'lab_values_admission_il6_in_pg_ml',
                                    'lab_values_admission_leukocytes_in_g_l',
                                    'lab_values_admission_bilirubin_in_mg_dl',
                                    'lab_values_admission_got_ast_in_u_l',
                                    'lab_values_admission_gpt_alt_in_u_l',
                                    'lab_values_admission_gamma-gt_in_u_l',
                                    'lab_values_admission_alkaline_phosphatase_in_u_l',
                                    #'lab_values_admission_lipase_in_u_l',
                                    #'lab_values_admission_inr',
                                    # 'lab_values_admission_quick _in_%',
                                    'lab_values_admission_calcium_in_mmol_l',
                                    # 'lab_values_admission_calcium_alb.-corrected_mmol_l',
                                    'lab_values_admission_triglycerides_in_mg_dl',
                                    'lab_values_admission_hematocrit',
                                    #'lab_values_admission_gfr_in_ml_min',
                                    'lab_values_admission_creatinine_in_mg_dl',
                                    #'lab_values_admission_urea_in_mg_dl',
                                    #'lab_values_admission_igg4_in_g_l'
                                    ]
# Fix names for use in model

# NUMERIC_LAB_FEATURE_NAMES_DAY3 = input_data.filter(like='lab_values_day_3').columns.tolist()

NUMERIC_LAB_FEATURE_NAMES_DAY3 = ['lab_values_day_3_crp_in_mg_dl',
                                #'lab_values_day_3_pct_in_ng_ml',
                                #'lab_values_day_3_il6_in_pg_ml',
                                #'lab_values_day_3_leukocytes_in_g_l',
                                'lab_values_day_3_bilirubin_in_mg_dl',
                                'lab_values_day_3_got_ast_in_u_l',
                                'lab_values_day_3_gpt_alt_in_u_l',
                                'lab_values_day_3_gamma-gt_in_u_l',
                                'lab_values_day_3_alkaline_phosphatase_in_u_l',
                                #'lab_values_day_3_lipase_in_u_l',
                                #'lab_values_day_3_inr',
                                #'lab_values_day_3_quick _in_%',
                                #'lab_values_day_3_calcium_in_mmol_l',
                                #'lab_values_day_3_calcium_alb.-corrected_mmol_l',
                                'lab_values_day_3_triglycerides_in_mg_dl',
                                'lab_values_day_3_hematocrit',
                                #'lab_values_day_3_gfr_in_ml_min',
                                'lab_values_day_3_creatinine_in_mg_dl',
                                #'lab_values_day_3_urea_in_mg_dl'
                                ]

NUMERIC_FEATURE_NAMES_ALL = NUMERIC_BASELINE_FEATURE_NAMES + NUMERIC_LAB_FEATURE_NAMES_ADMISSION + NUMERIC_LAB_FEATURE_NAMES_DAY3
ALL_FEATURE_NAMES_ADMISSION = TARGET_COLUMN_NAME + CATEGORICAL_FEATURE_NAMES + NUMERIC_BASELINE_FEATURE_NAMES + NUMERIC_LAB_FEATURE_NAMES_ADMISSION
ALL_FEATURE_NAMES_DAY3 = TARGET_COLUMN_NAME + CATEGORICAL_FEATURE_NAMES + NUMERIC_BASELINE_FEATURE_NAMES + NUMERIC_LAB_FEATURE_NAMES_DAY3

### Select and convert data types

In [ ]:
x_data = pd.DataFrame()

x_data_pre = input_data.copy()

x_data[TARGET_COLUMN_NAME] = x_data_pre[TARGET_COLUMN_NAME].astype('int')

x_data[CATEGORICAL_FEATURE_NAMES] = x_data_pre[CATEGORICAL_FEATURE_NAMES].fillna('').astype('category')
if len(CATEGORICAL_FEATURE_NAMES) > 0:
    x_data[CATEGORICAL_FEATURE_NAMES] = x_data[CATEGORICAL_FEATURE_NAMES].apply(lambda x: x.cat.codes).astype('int64')

x_data[NUMERIC_FEATURE_NAMES_ALL]= x_data_pre[NUMERIC_FEATURE_NAMES_ALL].fillna(float(np.nan)).astype('float64')

x_data[ECRP_GTE_3_DAYS] = x_data_pre[ECRP_GTE_3_DAYS].astype('bool')

x_data['dhc_in_mm'] = x_data_pre['eus_ercp_mrcp_or_as_dhc_width_in_mm']
x_data['clinical_data_stone_microlithiasis_sludge'] = x_data_pre['clinical_data_stone_microlithiasis_sludge']

x_data.dtypes

### Split data in training and test data

In [ ]:
train_df, test_df = train_test_split(x_data, test_size=0.2, random_state=42)

test_df_ecrp_gte_3_days = test_df[ECRP_GTE_3_DAYS]

test_df_pandas = test_df.copy()

# remove pass_dhc_col from test_df
test_df = test_df.drop(columns=['dhc_in_mm'])
train_df = train_df.drop(columns=['dhc_in_mm'])
test_df = test_df.drop(columns=['clinical_data_stone_microlithiasis_sludge'])

test_df_copy = test_df.copy()


test_labels = test_df_copy[TARGET_COLUMN_NAME].values
test_df_d3 = test_df_copy[ALL_FEATURE_NAMES_DAY3]
test_df_d3.columns = test_df_d3.columns.str.replace('baseline_characteristics_', '')
test_df_d3.columns = test_df_d3.columns.str.replace('lab_values_day_3_', '')
model_cols = test_df_d3.columns
test_df_d32 = test_df_d3.copy()



### Write model columns to csv

In [ ]:
model_cols_df = pd.DataFrame()
model_cols_df['model_cols'] = model_cols
model_cols_df['dtypes'] = test_df_d32.dtypes.astype('str').tolist()

# Write to csv
model_cols_df.to_csv("data/model_cols_df.csv", index=False)

#### Make train tf-dataset

In [ ]:
train_df

In [ ]:
train_df = train_df[ALL_FEATURE_NAMES_ADMISSION + ["clinical_data_stone_microlithiasis_sludge"]]
train_df.columns = train_df.columns.str.replace('baseline_characteristics_', '')
train_df.columns = train_df.columns.str.replace('lab_values_admission_', '')
#train_df = train_df[model_cols + ["clinical_data_stone_microlithiasis_sludge"]]

train_df.dtypes


In [ ]:
select_cols = model_cols 
select_cols = select_cols.append(pd.Index(["clinical_data_stone_microlithiasis_sludge"]))

train_df = train_df[select_cols]


### Impute missing values
Uses the MICE algorithm.

In [ ]:

# Impute training data
imputer = IterativeImputer(random_state=41)
imputed = imputer.fit_transform(train_df)
df_imputed = pd.DataFrame(imputed, columns=train_df.columns)

df_imputed['has_idc'] = df_imputed['has_idc'].astype('int64')

train_df_pd = df_imputed.copy()



### Oversample minority classes (sludge and microlithiasis)

In [ ]:
# Define minortiy class
is_sl_or_mic = df_imputed['clinical_data_stone_microlithiasis_sludge'].isin([2, 3, 6])
is_sl_or_mic = is_sl_or_mic.astype('int64')

In [ ]:
from imblearn.over_sampling import SMOTE
# Oversample the minority class using the SMOTE algorithm

# Apply SMOTE oversampling
smote = SMOTE(random_state=42)
train_df_oversampled, y_train_oversampled = smote.fit_resample(df_imputed, is_sl_or_mic)

print("Before oversampling" + str(df_imputed.shape) + "; after oversampling:" + str(train_df_oversampled.shape))


### Convert to tf dataset

In [ ]:
train_df = train_df_oversampled[model_cols]
train_df["sex"] = train_df["sex"].astype('int64')
train_df["nicotine_use"] = train_df["nicotine_use"].astype('int64')

train_cols = train_df.dtypes
train_df = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label="has_idc")


#### Make test tf-dataset

In [ ]:
test_labels = test_df[TARGET_COLUMN_NAME].values
test_df = test_df[ALL_FEATURE_NAMES_ADMISSION]
test_df.columns = test_df.columns.str.replace('baseline_characteristics_', '')
test_df.columns = test_df.columns.str.replace('lab_values_admission_', '')
test_df = test_df[model_cols]
test_cols = test_df.dtypes
test_df = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, label="has_idc")

In [ ]:
train_cols == test_cols

In [ ]:
test_cols

## Train the model - random forests BASELINE

In [ ]:
# Crete a tuner for model optimization on sensitivity (i.e. has_idc=1)

# model_rf = tfdf.keras.GradientBoostedTreesModel()
model_rf = tfdf.keras.RandomForestModel(
    #tuner=tuner,
    compute_oob_variable_importances=True,
    )
model_rf.fit(train_df)

### Evaluate the model
We adjust the threshold to 0.25 for sensitivity.

In [ ]:
model_rf.compile(metrics=["accuracy", "AUC", "TruePositives", "TrueNegatives", "FalsePositives", "FalseNegatives"])

In [ ]:
probs = model_rf.predict(test_df)
preds = probs > 0.25

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
confusion_matrix(test_labels, preds)

import matplotlib.pyplot as plt

cm = confusion_matrix(test_labels, preds)

disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()

**Figure.** Confusion matrix for the test set: True positives (TP), true negatives (TN), false positives (FP), false negatives (FN).

In [ ]:
# Write data for analysis in R to csv
test_df_pandas['pred_prob'] = probs
test_df_pandas['pred_idc'] = preds
test_df_pandas.to_csv("data/test_df_pandas.csv", index=False)

In [ ]:
evaluation = model_rf.evaluate(test_df, return_dict=True)
for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

#### Inspect the model

In [ ]:
print(model_rf.summary())

In [ ]:
inspector = model_rf.make_inspector()

inspector.evaluation()



In [ ]:
# Print all the variable importances
model_rf.summary()

# List the available variable importances
print(inspector.variable_importances().keys())

# Show a specific variable importance
# Each line is: (feature name, (index of the feature), importance score)
inspector.variable_importances()["MEAN_DECREASE_IN_ACCURACY"]


In [ ]:
print(f"Available variable importances:")
for importance in inspector.variable_importances().keys():
  print("\t", importance)


#### Plot feature importance
Report feature importance as Inverse mean minimum depth (INV_MEAN_MIN_DEPTH).


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

# Mean decrease in AUC of the class 1 vs the others.
variable_importance_metric = "INV_MEAN_MIN_DEPTH"
variable_importances = inspector.variable_importances()[variable_importance_metric]

# Extract the feature name and importance values.
#
# `variable_importances` is a list of <feature, importance> tuples.
feature_names = [vi[0].name for vi in variable_importances]
feature_importances = [vi[1] for vi in variable_importances]
# The feature are ordered in decreasing importance value.
feature_ranks = range(len(feature_names))

bar = plt.barh(feature_ranks, feature_importances, label=[str(x) for x in feature_ranks])
plt.yticks(feature_ranks, feature_names)
plt.gca().invert_yaxis()

# TODO: Replace with "plt.bar_label()" when available.
# Label each bar with values
for importance, patch in zip(feature_importances, bar.patches):
  plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{importance:.4f}", va="top")

plt.xlabel(variable_importance_metric)
plt.title("INV_MEAN_MIN_DEPTH")
plt.tight_layout()
plt.show()


### Write feature importances to csv

In [ ]:
df = pd.DataFrame()
df['feature_names'] = feature_names
df['feature_importances'] = feature_importances
df['feature_ranks'] = feature_ranks
df['variable_importance_metric'] = variable_importance_metric

# Add feature type (numeric or categorical) by checing, if feature name is in part in CATEGORICAL_FEATURE_NAMES
df['feature_type'] = 'numeric'
df['feature_type'] = np.where(df['feature_names'].str.contains('|'.join(CATEGORICAL_FEATURE_NAMES)), 'categorical', 'numeric')
# check for each name in df['feature_names'] if it is a substring of CATEGORICAL_FEATURE_NAMES
for i in df['feature_names']:
    i in ('|'.join(CATEGORICAL_FEATURE_NAMES))
    if i in ('|'.join(CATEGORICAL_FEATURE_NAMES)):
        df.loc[df['feature_names'] == i, 'feature_type'] = 'categorical'
df.to_csv("data/feature_importances.csv", index=False)

#### Plot the model

In [ ]:
# Tell dtreeviz about training data and model
model_features = [f.name for f in model_rf.make_inspector().features()]

model_features
classes = [0, 1]

train_df_pd['has_idc'] = train_df_pd['has_idc'].astype(int)

viz_cmodel = dtreeviz.model(model_rf,
                           tree_index=3,
                           X_train=train_df_pd[model_features],
                           y_train=train_df_pd['has_idc'],
                           feature_names=model_features,
                           target_name=TARGET_COLUMN_NAME,
                           class_names=classes)



##### Plot of complete model

In [ ]:
viz_cmodel.view()

##### First three levels of tree

In [ ]:
viz_cmodel.view(depth_range_to_display=[0,2])

##### Other plots

In [ ]:
x = train_df_pd[model_features].iloc[0]
viz_cmodel.view(x=x, show_just_path=True, scale=.75)

In [ ]:
train_df_pd.iloc[0]

In [ ]:
viz_cmodel.view(depth_range_to_display=[3,3], scale=1.5)

#### Plot the training logs

In [ ]:
import matplotlib.pyplot as plt

logs = model_rf.make_inspector().training_logs()

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")

plt.subplot(1, 2, 2)
plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Logloss (out-of-bag)")

plt.show()

## Test data for TFDF: Day 3

### Features

#### Test the model on day 3

In [ ]:
test_df_ecrp_gte_3_days.values

In [ ]:
# Subset test_df_d3 for only preds == True and ecrp_gte_3_days == True
d3 = test_df_ecrp_gte_3_days.values
tp = (np.logical_and(d3 == True, preds == False))
test_df_d32_keras = tfdf.keras.pd_dataframe_to_tf_dataset(test_df_d32[tp], label="has_idc")


probs_d3 = model_rf.predict(test_df_d32_keras)

probs_d3 = probs_d3 > 0.3

probs_d3 = probs_d3.astype(int)


from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
confusion_matrix(test_labels[tp], probs_d3)




import matplotlib.pyplot as plt

cm = confusion_matrix(test_labels[tp], probs_d3)

disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()


**Figure.** This figure shows the patients who did not receive an ERCP within 3 days of admission and who were classfied as 'no IDC' on admission. Note: the number of cases with IDC is very low!

In [ ]:
evaluation = model_rf.evaluate(test_df_d32_keras, return_dict=True)
for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")